<a href="https://colab.research.google.com/github/cojocarucosmin/AICourseDev/blob/main/CSV_and_Database_Assistant_with_LangChain_Agents_and_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required libraries

In [ ]:
!pip install -q openai langchain-openai langchain-community langchain-core langchain_experimental gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.0 MB/

## Create setup

In [ ]:
# only for Google Colab; please comment Kaggle part in this case
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# only for Kaggle; please comment Google Colab part in this case
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
#os.environ["OPENAI_API_KEY"] = user_secrets.get_secret("OPENAI_API_KEY")

In [ ]:
import os, openai
import pandas as pd
import gradio as gr

from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

## Get data from internet and save it locally

In [ ]:
os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)

--2024-06-27 14:24:42--  https://covidtracking.com/data/download/all-states-history.csv
Resolving covidtracking.com (covidtracking.com)... 172.67.183.132, 104.21.64.114, 2606:4700:3032::ac43:b784, ...
Connecting to covidtracking.com (covidtracking.com)|172.67.183.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./data/all-states-history.csv’

all-states-history.     [ <=>                ]   2.61M  --.-KB/s    in 0.08s   

2024-06-27 14:24:42 (34.4 MB/s) - ‘./data/all-states-history.csv’ saved [2738601]



## Explore the data

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20780 entries, 0 to 20779
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date                              20780 non-null  datetime64[ns]
 1   state                             20780 non-null  object        
 2   death                             20780 non-null  float64       
 3   deathConfirmed                    20780 non-null  float64       
 4   deathIncrease                     20780 non-null  int64         
 5   deathProbable                     20780 non-null  float64       
 6   hospitalized                      20780 non-null  float64       
 7   hospitalizedCumulative            20780 non-null  float64       
 8   hospitalizedCurrently             20780 non-null  float64       
 9   hospitalizedIncrease              20780 non-null  int64         
 10  inIcuCumulative                   20780 non-nu

In [ ]:
df

,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,inIcuCumulative,inIcuCurrently,negative,negativeIncrease,negativeTestsAntibody,negativeTestsPeopleAntibody,negativeTestsViral,onVentilatorCumulative,onVentilatorCurrently,positive,positiveCasesViral,positiveIncrease,positiveScore,positiveTestsAntibody,positiveTestsAntigen,positiveTestsPeopleAntibody,positiveTestsPeopleAntigen,positiveTestsViral,recovered,totalTestEncountersViral,totalTestEncountersViralIncrease,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease,month_year,year,month
0,2021-03-07,AK,305.0,0.0,0,0.0,1293.0,1293.0,33.0,0,0.0,0.0,0.0,0,0.0,0.0,1660758.0,0.0,2.0,56886.0,0.0,0,0,0.0,0.0,0.0,0.0,68693.0,0.0,0.0,0,1731628.0,0,0.0,0.0,0.0,0.0,0.0,0,1731628.0,0,2021-03,2021,3
1,2021-03-07,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,2676.0,0.0,1931711.0,2087,0.0,0.0,0.0,1515.0,0.0,499819.0,392077.0,408,0,0.0,0.0,0.0,0.0,0.0,295690.0,0.0,0,2323788.0,2347,0.0,0.0,119757.0,0.0,2323788.0,2347,0.0,0,2021-03,2021,3
2,2021-03-07,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,0.0,141.0,2480716.0,3267,0.0,0.0,2480716.0,1533.0,65.0,324818.0,255726.0,165,0,0.0,0.0,0.0,81803.0,0.0,315517.0,0.0,0,2736442.0,3380,0.0,0.0,0.0,481311.0,0.0,0,2736442.0,3380,2021-03,2021,3
3,2021-03-07,AS,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,2140.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2140.0,0,0.0,0.0,0.0,0.0,0.0,0,2140.0,0,2021-03,2021,3
4,2021-03-07,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,0.0,273.0,3073010.0,13678,0.0,0.0,0.0,0.0,143.0,826454.0,769935.0,1335,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,7908105.0,45110,580569.0,0.0,444089.0,0.0,3842945.0,14856,7908105.0,45110,2021-03,2021,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20775,2020-01-17,WA,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2020-01,2020,1
20776,2020-01-16,WA,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2020-01,2020,1
20777,2020-01-15,WA,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2020-01,2020,1
20778,2020-01-14,WA,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2020-01,2020,1


## Prepare the Langchain dataframe agent

In [ ]:
# Create instance of ChatOpenAI with the specified model and temperature setting
model = ChatOpenAI(temperature=0.5, model='gpt-4')

# Create agent to query the csv data
agent = create_pandas_dataframe_agent(llm=model,
                                      df=df,
                                      verbose=True,
                                      allow_dangerous_code=True,
                                      handle_parsing_errors=True)

## Start asking questions

In [ ]:
question = "how many rows are there for US?"

agent.invoke(question)

## Build template prompt

In [ ]:
prompt_prefix = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

prompt_suffix = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

In [ ]:
question = "How may patients were hospitalized in July 2020"

agent.invoke(prompt_prefix + question + prompt_suffix)



> Entering new AgentExecutor chain...
Thought: To answer the question, I need to find the number of patients hospitalized during July 2020. I will first set the pandas display options to show all the columns. Then, I will check the column names to see which column contains the information about hospitalizations.

Action: python_repl_ast
Action Input: df.info()<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20780 entries, 0 to 20779
Data columns (total 42 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date                              20780 non-null  datetime64[ns]
 1   state                             20780 non-null  object        
 2   death                             20780 non-null  float64       
 3   deathConfirmed                    20780 non-null  float64       
 4   deathIncrease                     20780 non-null  int64         
 5   deathProbable                   

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients were hospitalized during July 2020\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a section that starts with: "\n\nExplanation:\n".\nIn the explanation, 

In [ ]:
question = "How may patients deceased in July 2020 from Texas"

agent.invoke(prompt_prefix + question + prompt_suffix)



> Entering new AgentExecutor chain...
Thought: To answer the question, I need to filter the dataframe to get the rows corresponding to Texas for the month of July 2020. Then, I need to calculate the sum of the 'death' column for these rows. Before doing that, I need to set the pandas display option to show all columns and get the column names to confirm the column I will be working with.
Action: python_repl_ast
Action Input: 
```python
import pandas as pd
pd.set_option('display.max_columns', None)
df.columns
```Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveSc

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients deceased in July 2020 from Texas\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a section that starts with: "\n\nExplanation:\n".\nIn the explanation, me

## Create a Gradio interface to ask questions

In [ ]:
def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
#        history_langchain_format.append(AIMessage(content=ai))

    # Use the agent to generate the response
    response = agent.invoke(prompt_prefix + "Question: " + message + "Context: " + str(history_langchain_format) + prompt_suffix)

    return response['output']

# Launch the Gradio interface with the prediction function and subtitle
gr.ChatInterface(fn=predict, title="CSV Wizard", description="Answering questions from Covid statistics csv").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f6bd6d02bae46ca8c0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## SQL Agent - Connecting to SQL Database

In [ ]:
from sqlalchemy import create_engine
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

### Create SQLite database

In [ ]:
# Path to your SQLite database file
database_file_path = "./db/test.db"

# Create database if it doesn't exist
os.makedirs(os.path.dirname(database_file_path), exist_ok=True)

# Create an engine to connect to the SQLite database
engine = create_engine(f'sqlite:///{database_file_path}')

# convert month_year column to string as it has unsupported type for SQLite
df['month_year'] = df['month_year'].astype(str)

df.to_sql(
    'all_states_history',
    con=engine,
    if_exists='replace',
    index=False
)

20780

## Prepare SQL prompts

In [ ]:
sql_prefix_prompt = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [ ]:
sql_format_instructions = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input:
SELECT TOP (10) [death]
FROM covidtracking
WHERE state = 'TX' AND date LIKE '2020%'

Observation:
[(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state
is 'TX' and the date starts with '2020'. The query returned a list of tuples
with the number of deaths for each day in 2020. To answer the question,
I took the sum of all the deaths in the list, which is 27437.
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example

"""

## Build the SQL agent

In [ ]:
db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
model = ChatOpenAI(temperature=0.5, model='gpt-4')
toolkit = SQLDatabaseToolkit(db=db, llm=model)

agent_executor_SQL = create_sql_agent(
    prefix=sql_prefix_prompt,
    format_instructions = sql_format_instructions,
    llm=model,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

## Start asking questions

In [ ]:
question = """
How may patients were hospitalized by year and month?
"""

agent_executor_SQL.invoke(question)



> Entering new SQL Agent Executor chain...
I need to find the number of patients hospitalized by year and month. This information should be in a table that contains hospitalization data. I will first list all the tables in the database to identify the correct one.

Action: sql_db_list_tables
Action Input: ""all_states_historyThe table 'all_states_history' seems like it could contain the information I need. I will check the schema of this table to see if it has the necessary columns for hospitalization, year, and month.

Action: sql_db_schema
Action Input: "all_states_history"
CREATE TABLE all_states_history (
	date DATETIME, 
	state TEXT, 
	death FLOAT, 
	"deathConfirmed" FLOAT, 
	"deathIncrease" BIGINT, 
	"deathProbable" FLOAT, 
	hospitalized FLOAT, 
	"hospitalizedCumulative" FLOAT, 
	"hospitalizedCurrently" FLOAT, 
	"hospitalizedIncrease" BIGINT, 
	"inIcuCumulative" FLOAT, 
	"inIcuCurrently" FLOAT, 
	negative FLOAT, 
	"negativeIncrease" BIGINT, 
	"negativeTestsAntibody" FLOAT, 
	"n

{'input': '\nHow may patients were hospitalized by year and month?\n',
 'output': "The number of patients hospitalized by year and month is as follows:\n- January 2020: 0\n- February 2020: 0\n- March 2020: 32,221\n- April 2020: 784,533\n- May 2020: 2,561,872\n- June 2020: 4,038,095\n- July 2020: 5,535,849\n- August 2020: 7,723,933\n- September 2020: 8,799,572\n- October 2020: 10,414,102\n- November 2020: 12,283,586\n- December 2020: 16,262,903\n- January 2021: 20,134,520\n- February 2021: 20,714,612\n- March 2021: 5,405,727\n\nThis information was obtained by summing the 'hospitalized' column in the 'all_states_history' table for each year and month."}



> Entering new AgentExecutor chain...


In [ ]:
question = """
how many deaths were from Texas by year and month?
"""

agent_executor_SQL.invoke(question)



> Entering new SQL Agent Executor chain...
I need to query the database to get the number of deaths from Texas grouped by year and month. But first, I need to know the structure of the database and the names of the tables.

Action: sql_db_list_tables
Action Input: ""all_states_historyThe database has a table named 'all_states_history'. I need to check the schema of this table to understand its structure and identify the relevant columns for my query.

Action: sql_db_schema
Action Input: "all_states_history"
CREATE TABLE all_states_history (
	date DATETIME, 
	state TEXT, 
	death FLOAT, 
	"deathConfirmed" FLOAT, 
	"deathIncrease" BIGINT, 
	"deathProbable" FLOAT, 
	hospitalized FLOAT, 
	"hospitalizedCumulative" FLOAT, 
	"hospitalizedCurrently" FLOAT, 
	"hospitalizedIncrease" BIGINT, 
	"inIcuCumulative" FLOAT, 
	"inIcuCurrently" FLOAT, 
	negative FLOAT, 
	"negativeIncrease" BIGINT, 
	"negativeTestsAntibody" FLOAT, 
	"negativeTestsPeopleAntibody" FLOAT, 
	"negativeTestsViral" FLOAT, 
	"on

{'input': '\nhow many deaths were from Texas by year and month?\n',
 'output': "The number of deaths from Texas by year and month are as follows:\n- March 2020: 227\n- April 2020: 11,438\n- May 2020: 39,119\n- June 2020: 61,207\n- July 2020: 120,655\n- August 2020: 304,998\n- September 2020: 431,275\n- October 2020: 523,085\n- November 2020: 590,591\n- December 2020: 758,658\n- January 2021: 984,700\n- February 2021: 1,123,403\n- March 2021: 306,654\n\nI used the following query:\n\n```sql\nSELECT strftime('%Y-%m', date) as YearMonth, SUM(death) as TotalDeaths FROM all_states_history WHERE state = 'TX' GROUP BY YearMonth ORDER BY YearMonth\n```"}

## Create a Gradio interface to ask questions

In [ ]:
def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    # Use the agent to generate the response
    response = agent.invoke("Question: " + message + "Context: " + str(history_langchain_format))
    return response['output']

# Launch the Gradio interface with the prediction function and subtitle
gr.ChatInterface(fn=predict, title="SQL Wizard", description="Answering questions from Covid statistics Database").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d7639772fb65a1be06.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
